In [1]:
import h5py
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd

In [2]:
matFilename = '../2017-06-30_batchdata_updated_struct_errorcorrect.mat'
f = h5py.File(matFilename)

In [3]:
list(f.keys())

['#refs#', '#subsystem#', 'batch', 'batch_date']

In [4]:
batch = f['batch']

In [5]:
list(batch.keys())

['Vdlin',
 'barcode',
 'channel_id',
 'cycle_life',
 'cycles',
 'policy',
 'policy_readable',
 'summary']

In [6]:
%%time
num_cells = batch['summary'].shape[0]
bat_dict = {}
for i in range(num_cells):
    cl = f[batch['cycle_life'][i,0]].value
    ch_id = f[batch['channel_id'][i,0]].value
    policy = f[batch['policy_readable'][i,0]].value.tobytes()[::2].decode()
    summary_IR = np.hstack(f[batch['summary'][i,0]]['IR'][0,:].tolist())
    summary_QCharge = np.hstack(f[batch['summary'][i,0]]['QCharge'][0,:].tolist())
    summary_QDischarge = np.hstack(f[batch['summary'][i,0]]['QDischarge'][0,:].tolist())
    summary_Tavg = np.hstack(f[batch['summary'][i,0]]['Tavg'][0,:].tolist())
    summary_Tmin = np.hstack(f[batch['summary'][i,0]]['Tmin'][0,:].tolist())
    summary_Tmax = np.hstack(f[batch['summary'][i,0]]['Tmax'][0,:].tolist())
    summary_chargetime = np.hstack(f[batch['summary'][i,0]]['chargetime'][0,:].tolist())
    summary_cycle = np.hstack(f[batch['summary'][i,0]]['cycle'][0,:].tolist())
    summary = {'IR': summary_IR, 'QC': summary_QCharge, 'QD': summary_QDischarge, 'Tavg':
                summary_Tavg, 'Tmin': summary_Tmin, 'Tmax': summary_Tmax, 'chargetime': summary_chargetime,
                'cycle': summary_cycle}
    cycles = f[batch['cycles'][i,0]]
    cycle_dict = {}
#     for j in range(cycles['I'].shape[0]):
#         I = np.hstack((f[cycles['I'][j,0]].value))
#         Qc = np.hstack((f[cycles['Qc'][j,0]].value))
#         Qd = np.hstack((f[cycles['Qd'][j,0]].value))
#         Qdlin = np.hstack((f[cycles['Qdlin'][j,0]].value))
#         T = np.hstack((f[cycles['T'][j,0]].value))
#         Tdlin = np.hstack((f[cycles['Tdlin'][j,0]].value))
#         V = np.hstack((f[cycles['V'][j,0]].value))
#         dQdV = np.hstack((f[cycles['discharge_dQdV'][j,0]].value))
#         t = np.hstack((f[cycles['t'][j,0]].value))
#         cd = {'I': I, 'Qc': Qc, 'Qd': Qd, 'Qdlin': Qdlin, 'T': T, 'Tdlin': Tdlin, 'V':V, 'dQdV': dQdV, 't':t}
#         cycle_dict[str(j)] = cd
        
#     cell_dict = {'cycle_life': cl, 'ch_id':ch_id, 'charge_policy':policy, 'summary': summary, 'cycles': cycle_dict}
    cell_dict = {'cycle_life': cl, 'charge_policy':policy, 'summary': summary}
    key = 'b1c' + str(i)
    bat_dict[key]=   cell_dict

/home/tam/anaconda3/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


CPU times: user 789 ms, sys: 11.8 ms, total: 801 ms
Wall time: 799 ms


In [7]:
df = pd.DataFrame.from_dict(bat_dict)

In [8]:
df = df.T
df.head()

,cycle_life,charge_policy,summary
b1c0,[[300.0]],1C(4%)-6C,"{'IR': [0.017542101, 0.017505454, 0.017570902,..."
b1c1,[[148.0]],2C(10%)-6C,"{'IR': [0.017852232, 0.017582141, 0.018061424,..."
b1c2,[[438.0]],2C(2%)-5C,"{'IR': [0.017804198, 0.017894397, 0.017902471,..."
b1c3,[[335.0]],2C(7%)-5.5C,"{'IR': [0.017053859, 0.016825944, 0.016992988,..."
b1c4,[[444.0]],3.6C(22%)-5.5C,"{'IR': [0.017030258, 0.016936054, 0.016789593,..."


In [ ]:
df[df['charge_policy'] != ]

# Summary

In [9]:
type(df[['summary']])

pandas.core.frame.DataFrame

In [10]:
summary  = df[['summary']]

In [11]:
summary.head()

,summary
b1c0,"{'IR': [0.017542101, 0.017505454, 0.017570902,..."
b1c1,"{'IR': [0.017852232, 0.017582141, 0.018061424,..."
b1c2,"{'IR': [0.017804198, 0.017894397, 0.017902471,..."
b1c3,"{'IR': [0.017053859, 0.016825944, 0.016992988,..."
b1c4,"{'IR': [0.017030258, 0.016936054, 0.016789593,..."


In [23]:
summary['charge_policy'] =  df['charge_policy']

/home/tam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [57]:
df0 = pd.DataFrame()
count = 0
for i, j in summary['summary'].items():
    

    df1 = pd.DataFrame.from_dict(j)
    df1['cell'] = i
    df1['cycle_life'] = df['cycle_life'][i][0][0]  # take the cycle_life from main df and add it in the summary
    if len(df['charge_policy'][i].split('C')) !=3:
        df1['C1'] =  0 
    else:
        df1['C1'] = float(df['charge_policy'][i].split('C')[0])
    
    df1['C2'] = float(df['charge_policy'][i].split('-')[-1].split('C')[0])
    df1['Q1'] = int(df['charge_policy'][i].split('(')[-1].split(')')[0].split('%')[0])
    
    
    df0 = pd.concat([df0, df1])
       
print(df0.shape)

(24920, 13)


In [58]:
df0.head(10)


,IR,QC,QD,Tavg,Tmin,Tmax,chargetime,cycle,cell,cycle_life,C1,C2,Q1
0,0.017542,1.067561,1.068057,30.363545,28.999689,31.487127,10.332668,1.0,b1c0,300.0,1.0,6.0,4
1,0.017505,1.069810,1.070054,29.160560,26.734909,31.661930,10.180050,2.0,b1c0,300.0,1.0,6.0,4
2,0.017571,1.071287,1.071910,29.711384,28.086988,31.111683,10.308322,3.0,b1c0,300.0,1.0,6.0,4
3,0.017263,1.072626,1.072965,30.235232,28.464998,31.539034,10.276183,4.0,b1c0,300.0,1.0,6.0,4
4,0.017489,1.071483,1.071910,29.481769,27.695206,31.414228,10.263642,5.0,b1c0,300.0,1.0,6.0,4
5,0.017164,1.072955,1.073295,30.155729,28.780985,31.646267,10.192627,6.0,b1c0,300.0,1.0,6.0,4
6,0.017006,1.072841,1.073584,30.347167,28.949301,31.585262,10.268855,7.0,b1c0,300.0,1.0,6.0,4
7,0.017519,1.070809,1.072039,29.093283,27.254005,30.755775,10.207952,8.0,b1c0,300.0,1.0,6.0,4
8,0.017023,1.072972,1.073211,29.952229,28.634039,31.257359,10.285420,9.0,b1c0,300.0,1.0,6.0,4
9,0.016944,1.073020,1.073357,30.023263,27.823721,31.303814,10.189893,10.0,b1c0,300.0,1.0,6.0,4


In [59]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24920 entries, 0 to 744
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   IR          24920 non-null  float64
 1   QC          24920 non-null  float64
 2   QD          24920 non-null  float64
 3   Tavg        24920 non-null  float64
 4   Tmin        24920 non-null  float64
 5   Tmax        24920 non-null  float64
 6   chargetime  24920 non-null  float64
 7   cycle       24920 non-null  float64
 8   cell        24920 non-null  object 
 9   cycle_life  24920 non-null  float64
 10  C1          24920 non-null  float64
 11  C2          24920 non-null  float64
 12  Q1          24920 non-null  int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 2.7+ MB


In [60]:
df0['IR'] = df0['IR'].astype('float16')
df0['QC'] = df0['QC'].astype('float16')
df0['Qd'] = df0['QD'].astype('float16')
df0['Tavg'] = df0['Tavg'].astype('float16')
df0['Tmin'] = df0['Tmin'].astype('float16')
df0['Tmax'] = df0['Tmax'].astype('float16')
df0['chargetime'] = df0['chargetime'].astype('float16')

df0['cycle_life'] = df0['cycle_life'].astype('int16')

In [61]:
df0.head()

,IR,QC,QD,Tavg,Tmin,Tmax,chargetime,cycle,cell,cycle_life,C1,C2,Q1,Qd
0,0.017548,1.067383,1.068057,30.359375,29.000000,31.484375,10.335938,1.0,b1c0,300,1.0,6.0,4,1.068359
1,0.017502,1.069336,1.070054,29.156250,26.734375,31.656250,10.179688,2.0,b1c0,300,1.0,6.0,4,1.070312
2,0.017578,1.071289,1.071910,29.718750,28.093750,31.109375,10.304688,3.0,b1c0,300,1.0,6.0,4,1.072266
3,0.017258,1.072266,1.072965,30.234375,28.468750,31.531250,10.273438,4.0,b1c0,300,1.0,6.0,4,1.073242
4,0.017487,1.071289,1.071910,29.484375,27.687500,31.421875,10.265625,5.0,b1c0,300,1.0,6.0,4,1.072266


In [62]:
df0.shape

(24920, 14)

In [63]:
df0.to_pickle('summary_df_test.pkl')

In [64]:

pd.read_pickle('summary_df_test.pkl').tail()

,IR,QC,QD,Tavg,Tmin,Tmax,chargetime,cycle,cell,cycle_life,C1,C2,Q1,Qd
740,0.020279,0.836426,0.834485,33.34375,29.906250,38.21875,20.0,741.0,b1c47,713,6.0,3.0,60,0.834473
741,0.020279,0.834961,0.832141,33.21875,29.921875,38.25000,20.0,742.0,b1c47,713,6.0,3.0,60,0.832031
742,0.020370,0.832520,0.830499,33.28125,29.812500,38.34375,20.0,743.0,b1c47,713,6.0,3.0,60,0.830566
743,0.020401,0.830566,0.828347,33.37500,29.781250,38.28125,20.0,744.0,b1c47,713,6.0,3.0,60,0.828125
744,0.020432,0.828125,0.826203,33.40625,29.812500,38.37500,20.0,745.0,b1c47,713,6.0,3.0,60,0.826172
